In [13]:
%pip install jsonref
%pip install azure-ai-projects
%pip install python-dotenv
%pip install "opentelemetry"
%pip install "azure-core-tracing-opentelemetry"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement opentelemetry (from versions: none)
ERROR: No matching distribution found for opentelemetry
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install azure-ai-projects azure-identity opentelemetry-sdk azure-core-tracing-opentelemetry

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install opentelemetry-sdk
%pip install opentelemetry-api
%pip install opentelemetry-exporter-otlp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str="eastus2.api.azureml.ms;5c5037e5-d3f1-4e7b-b3a9-f6bf94902b30;rg-abeomorogbe-0793_ai;abeomorogbe-2923",
)

application_insights_connection_string = project_client.telemetry.get_connection_string()

In [15]:
from azure.core.settings import settings 

settings.tracing_implementation = "opentelemetry"

from azure.ai.inference.tracing import AIInferenceInstrumentor 

# Instrument AI Inference API 

AIInferenceInstrumentor().instrument()

In [3]:
import os
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails, OpenApiConnectionSecurityScheme, OpenApiConnectionAuthDetails
from dotenv import load_dotenv
load_dotenv(".env")


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

# project_client = AIProjectClient.from_connection_string(
#     credential=DefaultAzureCredential(),
#     conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
# )

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

application_insights_connection_string = project_client.telemetry.get_connection_string()

In [5]:
%pip install azure-monitor-opentelemetry

  Using cached opentelemetry_instrumentation_django-0.51b0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.51b0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_flask-0.51b0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_psycopg2-0.51b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_instrumentation_requests-0.51b0-py3-none-any.whl.metadata (2.7 kB)
  Using cached opentelemetry_instrumentation_urllib-0.51b0-py3-none-any.whl.metadata (3.5 kB)
  Using cached opentelemetry_instrumentation_urllib3-0.51b0-py3-none-any.whl.metadata (3.5 kB)
  Using cached opentelemetry_resource_detector_azure-0.1.5-py3-none-any.whl.metadata (5.3 kB)
  Using cached fixedint-0.1.6-py3-none-any.whl.metadata (4.8 kB)
  Using cached msrest-0.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached psutil-6.1.1-cp36-abi3-macosx_11_0_arm64.whl.metadata (22 kB)
  Using cached opentelemetry_instrument

In [6]:
from azure.monitor.opentelemetry import configure_azure_monitor
configure_azure_monitor(connection_string=application_insights_connection_string)

In [17]:
with open('./weather_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
connection_id = "/subscriptions/5c5037e5-d3f1-4e7b-b3a9-f6bf94902b30/resourceGroups/abe-ai-foundry/providers/Microsoft.MachineLearningServices/workspaces/project-demotldibnrwzlt3c/connections/cases_api"

auth = OpenApiAnonymousAuthDetails()
#auth = OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id))
# Initialize agent OpenAPI tool using the read in OpenAPI spec
openapi = OpenApiTool(name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth)

In [7]:
with open('./open_cases_openapi.json', 'r') as f:
    openapi_spec = jsonref.load(f)

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
connection_string = "/subscriptions/5c5037e5-d3f1-4e7b-b3a9-f6bf94902b30/resourceGroups/abe-ai-foundry/providers/Microsoft.MachineLearningServices/workspaces/project-demotldibnrwzlt3c/connections/cases_api"

auth = OpenApiAnonymousAuthDetails()
# auth = OpenApiConnectionAuthDetails(
#     security_scheme=OpenApiConnectionSecurityScheme(connection_id = connection_string)
# )
# Initialize agent OpenAPI tool using the read in OpenAPI spec
openapi = OpenApiTool(name="open_cases", spec=openapi_spec, description="Retrieves case data on every query.", auth=auth)

In [8]:
# Create agent with OpenAPI tool and process assistant run
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="cases-assistant-law",
    instructions="You are a helpful assistant",
    tools=openapi.definitions
)
print(f"Created agent, ID: {agent.id}")

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Get the case with: 'Water leaking from the apartment above'",
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Created agent, ID: asst_ONBbUp1jJMEDv6mjzxSUel3f
Created thread, ID: thread_QVoeJKlF1BLdAkRkXa39jXMw
Created message, ID: msg_SRwWoDvjwVdm1xsPSrpH0bMB
Run finished with status: RunStatus.COMPLETED


In [8]:
# Delete the assistant when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent


In [25]:
from pprint import pprint
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")
pprint(messages['data'][0]['content'][0]['text']['value'])

Messages: {'object': 'list', 'data': [{'id': 'msg_9ZzeUpdaUzlQxRxKiqyTV8jW', 'object': 'thread.message', 'created_at': 1740707256, 'assistant_id': 'asst_VxUK0FvEe5o02DM8oS5PARfF', 'thread_id': 'thread_h1Nex4jik7PeQWtBEkmYmjiD', 'run_id': 'run_rZaGCJNMegOQQP0r9WCnzhES', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'It seems there was an error while trying to retrieve the case data. Please try again later or adjust your query, and I will assist you further.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_fq9MsiN4PFHOy1uzulpBOQ5G', 'object': 'thread.message', 'created_at': 1740707251, 'assistant_id': None, 'thread_id': 'thread_h1Nex4jik7PeQWtBEkmYmjiD', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': "Get the case with: 'Water leaking from the apartment above'", 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_9ZzeUpdaUzlQxRxKiqyTV8jW', 'last_id': 'msg_fq9MsiN4PFHOy1uzulpBOQ5G', 'has_m